In [140]:
import json
import time

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [141]:
client = Elasticsearch()

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

100%|███████████████████████████████████████████████████████████████████████████████| 459M/459M [07:19<00:00, 1.05MB/s]


In [152]:
# def run_query_loop():
#     while True:
#         try:
#             handle_query()
#         except KeyboardInterrupt:
#             return

def handle_query(query):
#     query = ''

    embedding_start = time.time()
#     print([query])
    query_vector = embedder.encode(query)[0]
#     print(len(query_vector))
    embedding_time = time.time() - embedding_start

    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['text_vector']) + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }

    search_start = time.time()
    response = client.search(
        index='smartsearch',
        body={
            "size": 1,
            "query": script_query,
            "_source": {"includes": ["short_text","class"]}
        }
    )
    search_time = time.time() - search_start

    print()
    print("{} total hits.".format(response["hits"]["total"]["value"]))
    print("embedding time: {:.2f} ms".format(embedding_time * 1000))
    print("search time: {:.2f} ms".format(search_time * 1000))
    for hit in response["hits"]["hits"]:
        print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
        print(hit["_source"])
        print()

In [189]:
handle_query(['What is revenue of Infosys'])


107 total hits.
embedding time: 202.91 ms
search time: 5.57 ms
id: XRyiA3MBig-z7AKFPPWE, score: 1.423808
{'short_text': 'months  it noted  Infosys states even that has been sorted out  although it needs to bring the margin up to a more desirable level  the brokerage said   One needs to see how much of the rupee related gains are retained and how much spent Infosys is going to invest in', 'class': 'Infosys_Finance.docx'}



In [174]:
query_vector = embedder.encode('What has been published about medical care?')

In [103]:
Progress made in creating a vaccine
Use of artificial intelligence in testing and treatment
Mobilization of surge medical staff to address shortages in overwhelmed communities
Unemployment and job losses during the pandemic
Political impacts of the pandemic

43